In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../Problem 1 - Naive Bayes/DATA/adult.csv")

In [3]:
df = df.replace({" ?": np.nan})
df = df.fillna(df.mode().iloc[0])
df.columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]

In [4]:
targetcol = df.pop(14)
df.insert(0, 'target', targetcol)

In [5]:
cols = [1, 3, 5, 6, 7, 8, 9, 13, 'target']
for col in cols:
    df[col] = pd.Categorical(df[col]).codes

In [6]:
df = df.drop([2,4], axis =1)

In [7]:
df.columns = [0,1,2,3,4,5,6,7,8,9,10,11,12]
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,39,6,9,4,0,1,4,1,2174,0,40,38
1,0,50,5,9,2,3,0,4,1,0,0,13,38
2,0,38,3,11,0,5,1,4,1,0,0,40,38
3,0,53,3,1,2,5,0,2,1,0,0,40,38
4,0,28,3,9,2,9,5,2,0,0,0,40,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0,27,3,7,2,12,5,4,0,0,0,38,38
32557,1,40,3,11,2,6,0,4,1,0,0,40,38
32558,0,58,3,11,6,0,4,4,0,0,0,40,38
32559,0,22,3,11,4,0,3,4,1,0,0,20,38


In [8]:
randlist=[1223, 91, 760, 1096, 670, 715, 373, 69, 538, 455]

In [9]:
train_array = [1]*10
test_array = [1]*10
X_train= [1]*10
X_tr= [1]*10
X_test= [1]*10
y_train= [1]*10
y_test = [1]*10
for i in range(10):
    train_array[i] = df.sample(frac=0.67, random_state = randlist[i])
    test_array[i] = df.drop(train_array[i].index)
    trainData_Split = train_array[i].to_numpy()
    testData_Split = test_array[i].to_numpy()
    n_samples,n_features = trainData_Split.shape
    n_features -= 1
    X_train[i] = trainData_Split[:,1:]
    y_train[i] = trainData_Split[:,0]
    X_test[i] = testData_Split[:,1:]
    y_test[i] = testData_Split[:,0]

In [10]:
class NaiveBayes:
    
    def fit(self, X, y, alpha=1.0):
        
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)
        
        for c in self._classes:
            X_c = X[c==y]
            self._mean[c,:] = X_c.mean(axis=0)
            self._var[c,:] = X_c.var(axis=0)
            self._priors[c] = (X_c.shape[0] + alpha) / (n_samples + alpha*n_classes)
        
        self.alpha = alpha
    
    
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return y_pred
    
    
    def _predict(self, x):
        posteriors = []
        
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditional = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        
        return self._classes[np.argmax(posteriors)]
    
    
    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-(x-mean)**2/(2*(var + self.alpha)))
        denominator = np.sqrt(2*np.pi*(var + self.alpha))
        
        return numerator/denominator

In [11]:
nb = [1]*10
nb_l = [1]*10

for i in range(10):
    nb[i] = NaiveBayes()
    nb[i].fit(X_train[i], y_train[i], alpha=0)
    
    nb_l[i] = NaiveBayes()
    nb_l[i].fit(X_train[i], y_train[i], alpha=1)

In [12]:
preds = [1]*10
predsL = [1]*10

for i in range(10):
    preds[i] = nb[i].predict(X_test[i])
    predsL[i] = nb_l[i].predict(X_test[i])

In [13]:
def computeMetrics(y_actual, y_predicted):
    y_predicted = np.array(y_predicted)
    tp = np.sum((y_actual == 1) & (y_predicted == 1))
    tn = np.sum((y_actual == 0) & (y_predicted == 0))
    fp = np.sum((y_actual == 0) & (y_predicted == 1))        
    fn = np.sum((y_actual == 1) & (y_predicted == 0)) 
    accuracy = 100*(np.sum(y_actual == y_predicted)/len(y_actual))
    precision = tp/np.sum(tp+fp)
    recall = tp/np.sum(tp+fn)
    f1score = (2*precision*recall)/(precision+recall)
    return accuracy, precision, recall, f1score

In [14]:
acc = [1]*10
prec = [1]*10
rec = [1]*10
f1 = [1]*10

accL = [1]*10
precL = [1]*10
recL = [1]*10
f1L = [1]*10

for i in range(10):
    acc[i], prec[i], rec[i], f1[i] = computeMetrics(y_test[i], preds[i])
    accL[i], precL[i], recL[i], f1L[i] = computeMetrics(y_test[i], predsL[i])
    print("Metrics for test-train split {} with no smoothing".format(i))
    print(acc[i], prec[i], rec[i], f1[i])
    print("\n")
    print("Metrics for test-train split {} with Laplace smoothing".format(i))
    print(accL[i], precL[i], recL[i], f1L[i])
    print("\n##############################################################################################################################\n")

Metrics for test-train split 0 with no smoothing
79.77664029781295 0.6851698425849213 0.31564885496183204 0.4321923177423569


Metrics for test-train split 0 with Laplace smoothing
79.5253606328525 0.6701782820097245 0.31564885496183204 0.4291645044110015

##############################################################################################################################

Metrics for test-train split 1 with no smoothing
80.26989297347603 0.7052722558340536 0.31445086705202313 0.4349680170575693


Metrics for test-train split 1 with Laplace smoothing
80.01861330851558 0.6895093062605753 0.3140655105973025 0.43155943870796926

##############################################################################################################################

Metrics for test-train split 2 with no smoothing
79.72080037226617 0.6561461794019934 0.30919765166340507 0.42032455440276667


Metrics for test-train split 2 with Laplace smoothing
79.5160539785947 0.6446078431372549 0.308806262

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [16]:
lr = [1]*10
knn = [1]*10

for i in range(10):
    lr[i] = LogisticRegression()
    knn[i] = KNeighborsClassifier(n_neighbors=3)

In [17]:
for i in range(10):
    lr[i].fit(X_train[i], y_train[i])
    knn[i].fit(X_train[i], y_train[i])

In [18]:
lrpreds = [1]*10
knnpreds = [1]*10
for i in range(10):
    knnpreds[i] = knn[i].predict(X_test[i])
    lrpreds[i] = lr[i].predict(X_test[i])

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
for i in range(10):
    print("Metrics for test-train split {} with Log-Reg".format(i+1))
    print(accuracy_score(y_test[i], lrpreds[i]), 
         precision_score(y_test[i], lrpreds[i]),
         recall_score(y_test[i], lrpreds[i]),
         f1_score(y_test[i], lrpreds[i]))
    print("\n")
    print("Metrics for test-train split {} with KNN".format(i+1))
    print(accuracy_score(y_test[i], knnpreds[i]), 
         precision_score(y_test[i], knnpreds[i]),
         recall_score(y_test[i], knnpreds[i]),
         f1_score(y_test[i], knnpreds[i]))
    print("\n##############################################################################################################################\n")

Metrics for test-train split 1 with Log-Reg
0.8001861330851558 0.7044079515989629 0.3110687022900763 0.43155943870796926


Metrics for test-train split 1 with KNN
0.8175895765472313 0.6325301204819277 0.601145038167939 0.6164383561643836

##############################################################################################################################

Metrics for test-train split 2 with Log-Reg
0.8024197301070265 0.7088495575221239 0.3086705202312139 0.43006711409395976


Metrics for test-train split 2 with KNN
0.8265239646347138 0.6442953020134228 0.6289017341040463 0.6365054602184086

##############################################################################################################################

Metrics for test-train split 3 with Log-Reg
0.8006514657980456 0.6858685868586859 0.298238747553816 0.41571194762684116


Metrics for test-train split 3 with KNN
0.8233597021870638 0.6360248447204969 0.601174168297456 0.6181086519114688

###########################

In [21]:
lracc = [1]*10
lrprec = [1]*10
lrrec = [1]*10
lrf1 = [1]*10

knnacc = [1]*10
knnprec = [1]*10
knnrec = [1]*10
knnf1 = [1]*10

for i in range(10):
    lracc[i], lrprec[i], lrrec[i], lrf1[i] = computeMetrics(y_test[i], lrpreds[i])
    knnacc[i], knnprec[i], knnrec[i], knnf1[i] = computeMetrics(y_test[i], knnpreds[i])
    print("Metrics for test-train split {} LR".format(i))
    print(lracc[i], lrprec[i], lrrec[i], lrf1[i])
    print("\n")
    print("Metrics for test-train split {} KNN".format(i))
    print(knnacc[i], knnprec[i], knnrec[i], knnf1[i])
    print("\n##############################################################################################################################\n")

Metrics for test-train split 0 LR
80.01861330851558 0.7044079515989629 0.3110687022900763 0.43155943870796926


Metrics for test-train split 0 KNN
81.75895765472313 0.6325301204819277 0.601145038167939 0.6164383561643836

##############################################################################################################################

Metrics for test-train split 1 LR
80.24197301070265 0.7088495575221239 0.3086705202312139 0.43006711409395976


Metrics for test-train split 1 KNN
82.65239646347138 0.6442953020134228 0.6289017341040463 0.6365054602184086

##############################################################################################################################

Metrics for test-train split 2 LR
80.06514657980456 0.6858685868586859 0.298238747553816 0.41571194762684116


Metrics for test-train split 2 KNN
82.33597021870638 0.6360248447204969 0.601174168297456 0.6181086519114688

##############################################################################

In [23]:
acdf = pd.DataFrame(acc, columns = ['Naive Bayes No Smoothing'])
acdf

,Naive Bayes No Smoothing
0,79.776640
1,80.269893
2,79.720800
3,79.432294
4,79.413681
5,80.055840
6,79.916240
7,79.702187
8,79.832480
9,79.888320


In [24]:
acdf['Naive Bayes Laplace Smoothing'] = accL
acdf['Logistic Regression'] = lracc
acdf['K Nearest Neighbours'] = knnacc
acdf

,Naive Bayes No Smoothing,Naive Bayes Laplace Smoothing,Logistic Regression,K Nearest Neighbours
0,79.776640,79.525361,80.018613,81.758958
1,80.269893,80.018613,80.241973,82.652396
2,79.720800,79.516054,80.065147,82.335970
3,79.432294,79.181014,79.702187,82.577943
4,79.413681,79.208934,79.813867,82.531410
5,80.055840,79.851094,80.474639,83.154956
6,79.916240,79.581201,80.279200,82.978129
7,79.702187,79.367148,79.795254,82.633783
8,79.832480,79.534667,79.776640,82.140530
9,79.888320,79.627734,79.879013,82.894369


In [25]:
prdf = pd.DataFrame(prec, columns = ['Naive Bayes No Smoothing'])
prdf['Naive Bayes Laplace Smoothing'] = precL
prdf['Logistic Regression'] = lrprec
prdf['K Nearest Neighbours'] = knnprec

prdf

,Naive Bayes No Smoothing,Naive Bayes Laplace Smoothing,Logistic Regression,K Nearest Neighbours
0,0.685170,0.670178,0.704408,0.632530
1,0.705272,0.689509,0.708850,0.644295
2,0.656146,0.644608,0.685869,0.636025
3,0.663407,0.648885,0.686308,0.648812
4,0.675314,0.663106,0.711152,0.652385
5,0.670311,0.657873,0.712665,0.652353
6,0.662369,0.643922,0.687925,0.651993
7,0.677665,0.657119,0.692308,0.648006
8,0.668664,0.651085,0.672291,0.628503
9,0.653878,0.638957,0.658348,0.642622


In [26]:
recdf = pd.DataFrame(rec, columns = ['Naive Bayes No Smoothing'])
recdf['Naive Bayes Laplace Smoothing'] = recL
recdf['Logistic Regression'] = lrrec
recdf['K Nearest Neighbours'] = knnrec

recdf

,Naive Bayes No Smoothing,Naive Bayes Laplace Smoothing,Logistic Regression,K Nearest Neighbours
0,0.315649,0.315649,0.311069,0.601145
1,0.314451,0.314066,0.308671,0.628902
2,0.309198,0.308806,0.298239,0.601174
3,0.311758,0.312141,0.303332,0.617005
4,0.306727,0.306727,0.295705,0.613455
5,0.312794,0.313187,0.295918,0.620094
6,0.320968,0.320578,0.315892,0.613042
7,0.307958,0.308727,0.297578,0.618608
8,0.304959,0.304569,0.295588,0.613042
9,0.309881,0.309881,0.302372,0.616206


In [27]:
f1df = pd.DataFrame(f1, columns = ['Naive Bayes No Smoothing'])
f1df['Naive Bayes Laplace Smoothing'] = f1L
f1df['Logistic Regression'] = lrf1
f1df['K Nearest Neighbours'] = knnf1

f1df

,Naive Bayes No Smoothing,Naive Bayes Laplace Smoothing,Logistic Regression,K Nearest Neighbours
0,0.432192,0.429165,0.431559,0.616438
1,0.434968,0.431559,0.430067,0.636505
2,0.420325,0.417571,0.415712,0.618109
3,0.424179,0.421515,0.420717,0.632509
4,0.421850,0.419439,0.417718,0.632321
5,0.426545,0.424355,0.418192,0.635815
6,0.432404,0.428050,0.432968,0.631918
7,0.423473,0.420089,0.416241,0.632966
8,0.418879,0.415004,0.410632,0.620676
9,0.420488,0.417354,0.414410,0.629136


In [28]:
df1 = acdf
df1

,Naive Bayes No Smoothing,Naive Bayes Laplace Smoothing,Logistic Regression,K Nearest Neighbours
0,79.776640,79.525361,80.018613,81.758958
1,80.269893,80.018613,80.241973,82.652396
2,79.720800,79.516054,80.065147,82.335970
3,79.432294,79.181014,79.702187,82.577943
4,79.413681,79.208934,79.813867,82.531410
5,80.055840,79.851094,80.474639,83.154956
6,79.916240,79.581201,80.279200,82.978129
7,79.702187,79.367148,79.795254,82.633783
8,79.832480,79.534667,79.776640,82.140530
9,79.888320,79.627734,79.879013,82.894369


In [33]:
df1=df1.append(prdf)
df1=df1.append(recdf)
df1=df1.append(f1df)
df1

,Naive Bayes No Smoothing,Naive Bayes Laplace Smoothing,Logistic Regression,K Nearest Neighbours
0,79.776640,79.525361,80.018613,81.758958
1,80.269893,80.018613,80.241973,82.652396
2,79.720800,79.516054,80.065147,82.335970
3,79.432294,79.181014,79.702187,82.577943
4,79.413681,79.208934,79.813867,82.531410
5,80.055840,79.851094,80.474639,83.154956
6,79.916240,79.581201,80.279200,82.978129
7,79.702187,79.367148,79.795254,82.633783
8,79.832480,79.534667,79.776640,82.140530
9,79.888320,79.627734,79.879013,82.894369


In [34]:
df1.to_excel("../Problem 1 - Naive Bayes/Metrics.xlsx")